<a href="https://colab.research.google.com/github/yagizterzi/MyPyhtonProjects/blob/main/BraTS2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nilearn # install the nilearn library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Conv2D, UpSampling2D, MaxPooling2D, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import glob
import skimage.io as io
import skimage.color as color
import random as r
import math
from nilearn import plotting # import the plotting module from nilearn
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 25.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data_from_folders(train_folder, validation_folder):
    X_train, Y_train = [], []
    X_val, Y_val = [], []

    # Eğitim verileri
    for filename in os.listdir(train_folder):
        if filename.endswith('.nii'):
            img_path = os.path.join(train_folder, filename)
            img_data = nib.load(img_path).get_fdata()
            img_data = img_data[:, :, img_data.shape[2] // 2]  # Orta dilimi al
            img_data = cv.resize(img_data, (128, 128))  # Boyutlandır
            img_data = img_data.astype(np.float32) / np.max(img_data)  # Normalizasyon
            X_train.append(img_data)

            # Maske yükleme
            mask_path = os.path.join(train_folder, 'masks', filename)  # Maske klasörü
            if os.path.exists(mask_path):
                mask_data = nib.load(mask_path).get_fdata()
                mask_data = mask_data[:, :, mask_data.shape[2] // 2]  # Orta dilimi al
                mask_data = cv.resize(mask_data, (128, 128))  # Boyutlandır
                Y_train.append(mask_data)

    # Doğrulama verileri
    for filename in os.listdir(validation_folder):
        if filename.endswith('.nii'):
            img_path = os.path.join(validation_folder, filename)
            img_data = nib.load(img_path).get_fdata()
            img_data = img_data[:, :, img_data.shape[2] // 2]  # Orta dilimi al
            img_data = cv.resize(img_data, (128, 128))  # Boyutlandır
            img_data = img_data.astype(np.float32) / np.max(img_data)  # Normalizasyon
            X_val.append(img_data)

            # Maske yükleme
            mask_path = os.path.join(validation_folder, 'masks', filename)
            if os.path.exists(mask_path):
                mask_data = nib.load(mask_path).get_fdata()
                mask_data = mask_data[:, :, mask_data.shape[2] // 2]  # Orta dilimi al
                mask_data = cv.resize(mask_data, (128, 128))  # Boyutlandır
                Y_val.append(mask_data)

    return np.array(X_train), np.array(Y_train), np.array(X_val), np.array(Y_val)

# Klasör yollarını belirt (Google Drive'daki yollar)
train_folder = '/content/drive/My Drive/path_to_your_train_folder'  # Eğitim verileri
validation_folder = '/content/drive/My Drive/path_to_your_validation_folder'  # Doğrulama verileri

X_train, Y_train, X_val, Y_val = load_data_from_folders(train_folder, validation_folder)


print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_val shape:', X_val.shape)
print('Y_val shape:', Y_val.shape)

In [ ]:
def unet_model(input_shape):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), padding='same', activation='relu')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), padding='same', activation='relu')(p2)
    p3 = MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = Conv2D(512, (3, 3), padding='same', activation='relu')(p3)

    # Decoder
    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c3])
    c5 = Conv2D(256, (3, 3), padding='same', activation='relu')(u5)

    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c2])
    c6 = Conv2D(128, (3, 3), padding='same', activation='relu')(u6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c1])
    c7 = Conv2D(64, (3, 3), padding='same', activation='relu')(u7)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Modeli tanımla
input_shape = (128, 128, 1)  # Giriş boyutu
model = unet_model(input_shape)

# Modeli derle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğit
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train.reshape(-1, 128, 128, 1), Y_train.reshape(-1, 128, 128, 1),
                    validation_data=(X_val.reshape(-1, 128, 128, 1), Y_val.reshape(-1, 128, 128, 1)),
                    epochs=20, batch_size=16, callbacks=[early_stopping])